# <strong> A - Importation de modules </strong>

In [1]:
############################################
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import cluster , metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn import manifold, decomposition
from tensorflow import keras
import tensorflow as tf

############################################
import seaborn as sns
#%matplotlib inline
#%matplotlib notebook
sns.set()

###########################################
import import_ipynb
from P6_01_My_functions_Kengni_zanguim_Brice_062022 import *

##########################################
PATH_IMAGE =  "/media/brice_kengni_zanguim/Samsung_T5/Téléchargements/photos"

importing Jupyter notebook from P6_01_My_functions_Kengni_zanguim_Brice_062022.ipynb


# <strong> B - Importation de données </strong>

In [2]:
picts = pd.read_csv("photos_leger.csv")

In [3]:
label_encoder = preprocessing.LabelEncoder().fit(picts.label)

In [4]:
sub_idx , _ = get_train_test_index( picts, 'label', (800,1) )

In [5]:
picts = picts.loc[sub_idx,:]

In [6]:
print( f"Shape : {picts.shape} " )

Shape : (4000, 2) 


In [7]:
picts = load_images( picts )

In [8]:
print( f"Shape : {picts.shape} " )

Shape : (4000, 3) 


In [9]:
picts = process_images( picts , 'images', 'RGEC', (90,90) )

In [10]:
train_idx, test_idx = get_train_test_index( picts,"label" , (0.8 ) )

In [11]:
picts ,picts_test = picts.loc[train_idx,:], picts.loc[test_idx,:]

In [12]:
print( f"- train shape : {picts.shape} \n- Test shape : {picts_test.shape}" )

- train shape : (3200, 4) 
- Test shape : (800, 4)


In [13]:
X = np.asarray( [ i  for i in picts.images_process] )/255.
X_test = np.asarray( [ i for i in picts_test.images_process] )/255.

In [14]:
True_label = label_encoder.transform(picts.label)
True_label_test =label_encoder.transform(picts_test.label)

In [15]:
Y = True_label.astype(float)
Y_test = True_label_test.astype(float)
#Y , Y_test = Y.reshape((1,Y.size)) , Y_test.reshape((1,Y_test.size))

# <strong> J -  Convolutional Neural Network </strong>

In [16]:
def generate_neural_network () :
    model = keras.Sequential([
    
    # Première couche de convolution
    keras.layers.Conv2D(filters=60, kernel_size= 3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3) , strides=2), 
        
    # Première couche de convolution
    keras.layers.Conv2D(filters=120, kernel_size= 3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3) , strides=2),
        
    # Fully Connected Classifier ( couche de réseau de Neuronnes denses )
    keras.layers.Flatten(),
    keras.layers.Dense(524, activation="relu"),
    keras.layers.Dense(524, activation="relu"),
    keras.layers.Dense(5, activation="softmax"),
    
    ])
        
    return model

### <strong> J.1 -  Convertion des images en array </strong>

In [17]:
CNN = generate_neural_network()

CNN.compile( optimizer= "adam", loss= keras.losses.sparse_categorical_crossentropy ,metrics = ["accuracy"])

CNN.fit( X , Y ,  epochs = 12 , batch_size = 80 , validation_split = 0.25 , use_multiprocessing = True )

2022-07-31 13:54:06.581676: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-31 13:54:06.583713: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/12
30/30 [==============================] - 24s 796ms/step - loss: 2.0125 - accuracy: 0.2525 - val_loss: 1.3809 - val_accuracy: 0.4200
Epoch 2/12
30/30 [==============================] - 25s 823ms/step - loss: 1.3081 - accuracy: 0.4696 - val_loss: 1.2793 - val_accuracy: 0.4638
Epoch 3/12
30/30 [==============================] - 24s 791ms/step - loss: 1.1526 - accuracy: 0.5529 - val_loss: 1.1626 - val_accuracy: 0.5412
Epoch 4/12
30/30 [==============================] - 21s 718ms/step - loss: 0.9441 - accuracy: 0.6408 - val_loss: 1.1760 - val_accuracy: 0.5562
Epoch 5/12
30/30 [==============================] - 21s 715ms/step - loss: 0.7740 - accuracy: 0.7163 - val_loss: 1.1452 - val_accuracy: 0.5788
Epoch 6/12
30/30 [==============================] - 21s 696ms/step - loss: 0.6117 - accuracy: 0.7758 - val_loss: 1.2929 - val_accuracy: 0.5525
Epoch 7/12
30/30 [==============================] - 21s 713ms/step - loss: 0.4108 - accuracy: 0.8571 - val_loss: 1.4113 - val_accuracy: 0.5750

# <strong> K -  Transfert Learning </strong>

In [146]:
modele_de_base = tf.keras.applications.MobileNet( include_top= False , input_shape= X[0].shape )

In [147]:
entrainer_le_modèle_de_base = ( True , 50 )

modele_de_base.trainable = False

if entrainer_le_modèle_de_base[0] :
    for layer in modele_de_base.layers[-entrainer_le_modèle_de_base[1] : ] :
        layer.trainable = True

In [148]:
#modele_de_base.summary()

In [149]:
couche_a_entrainer = keras.Sequential()

#couche_a_entrainer.add( keras.layers.MaxPool2D() )
couche_a_entrainer.add( keras.layers.Flatten() )

couche_a_entrainer.add( keras.layers.Dense(25, activation="tanh"),)
couche_a_entrainer.add( keras.layers.Dense(5, activation="softmax"),)

In [150]:
CNN_transfert_learning = keras.Sequential( [modele_de_base, couche_a_entrainer] )

In [151]:
CNN_transfert_learning.compile( optimizer= "adam", loss= keras.losses.sparse_categorical_crossentropy ,metrics = ["accuracy"])

CNN_transfert_learning.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 2, 2, 1024)       3228864   
 nal)                                                            
                                                                 
 sequential_28 (Sequential)  (None, 5)                 102555    
                                                                 
Total params: 3,331,419
Trainable params: 102,555
Non-trainable params: 3,228,864
_________________________________________________________________


In [152]:
pd.DataFrame().drop

<bound method DataFrame.drop of Empty DataFrame
Columns: []
Index: []>

In [153]:
CNN_transfert_learning.fit( X , Y ,  epochs = 16, batch_size= 300, validation_split=0.15, use_multiprocessing=True)

Epoch 1/16
10/10 [==============================] - 9s 835ms/step - loss: 1.3922 - accuracy: 0.4592 - val_loss: 0.9678 - val_accuracy: 0.6479
Epoch 2/16
10/10 [==============================] - 8s 809ms/step - loss: 0.7413 - accuracy: 0.7482 - val_loss: 0.7186 - val_accuracy: 0.7625
Epoch 3/16
10/10 [==============================] - 8s 779ms/step - loss: 0.5778 - accuracy: 0.8074 - val_loss: 0.7149 - val_accuracy: 0.7750
Epoch 4/16
10/10 [==============================] - 8s 820ms/step - loss: 0.4729 - accuracy: 0.8386 - val_loss: 0.7125 - val_accuracy: 0.7729
Epoch 5/16
10/10 [==============================] - 8s 791ms/step - loss: 0.4003 - accuracy: 0.8691 - val_loss: 0.7043 - val_accuracy: 0.7708
Epoch 6/16
10/10 [==============================] - 8s 811ms/step - loss: 0.3432 - accuracy: 0.9007 - val_loss: 0.7029 - val_accuracy: 0.7729
Epoch 7/16
10/10 [==============================] - 8s 767ms/step - loss: 0.2963 - accuracy: 0.9210 - val_loss: 0.6801 - val_accuracy: 0.7833
Epoch 

In [154]:
a = [np.argmax(i ) for i in CNN_transfert_learning.predict( X_test ) ]
metrics.adjusted_rand_score(Y_test , a)

0.5221333092564904

In [155]:
metrics.accuracy_score( a , Y_test )

0.7725